In [ ]:
!pip install selenium

In [ ]:
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import UnexpectedAlertPresentException
import pandas as pd

In [ ]:
# driver 세팅
options = wd.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# driver=wd.Chrome(executable_path='chromedriver', options=options)
url = 'https://sugang.snu.ac.kr/'
# driver.implicitly_wait(3)
# driver.get(url)
# driver.implicitly_wait(5)

In [ ]:
# - * - 검색창 활성화 - * - #
# iframe 선택
# frame = driver.find_element(by=By.XPATH, value = '//*[@id="main"]')
# driver.switch_to.frame(frame)
# driver.implicitly_wait(3)

In [ ]:

# # 돋보기 아이콘 클릭
# driver.find_element(by=By.XPATH, value='//*[@id="header"]/div[2]/div[1]/button[1]').click()
# driver.implicitly_wait(4)

# # 상세 조건 버튼 클릭
# driver.find_element(by=By.XPATH, value='//*[@id="HD100"]/div/div/div[1]/fieldset/div[3]/button[2]').click()
# driver.implicitly_wait(10)

In [ ]:

# # 이전학기 조건 보기 클릭
# driver.find_element(by=By.XPATH, value='//*[@id="HD100"]/div/div/div[3]/div[2]/div/fieldset/button').click()
# driver.implicitly_wait(2)

In [ ]:
# 검색조건 명시 -> 리스트 형태로 다 때려 넣으면 한번에 실행해줌!!
years = ['2022']
semesters = ['2학기'] #'2학기'
query_keywords = ['AI']
#level = '대학원' # '전체' # '학사'

In [ ]:
for query_keyword in query_keywords:
    for year in years:
        for semester in  semesters:
            driver=wd.Chrome(executable_path='chromedriver', options=options)
            driver.implicitly_wait(3)
            driver.get(url)
            driver.implicitly_wait(5)
            frame = driver.find_element(by=By.XPATH, value = '//*[@id="main"]')
            driver.switch_to.frame(frame)
            driver.implicitly_wait(3)
            
            # 돋보기 아이콘 클릭
            driver.find_element(by=By.XPATH, value='//*[@id="header"]/div[2]/div[1]/button[1]').click()
            driver.implicitly_wait(4)
            # 상세 조건 버튼 클릭
            driver.find_element(by=By.XPATH, value='//*[@id="HD100"]/div/div/div[1]/fieldset/div[3]/button[2]').click()
            driver.implicitly_wait(10)


            # 이전학기 조건 보기 클릭
            driver.find_element(by=By.XPATH, value='//*[@id="HD100"]/div/div/div[3]/div[2]/div/fieldset/button').click()
            driver.implicitly_wait(2)

            # 년도 변경
            select = Select(driver.find_element(by=By.XPATH, value='//*[@id="hSrchOpenSchyy"]'))
            select.select_by_visible_text(year)

            # 학기 변경
            select = Select(driver.find_element(by=By.XPATH, value='//*[@id="hSrchOpenShtm"]'))
            select.select_by_visible_text(semester)

            # 검색어 입력
            search = driver.find_element(By.XPATH, value='//*[@id="hMobileTotalSearch"]')
            search.clear()
            search.send_keys(query_keyword)

            # # 학부 포함 여부
            # select = Select(driver.find_element(by=By.XPATH, value='//*[@id="hSrchOpenSubmattCorsFg"]'))
            # select.select_by_visible_text(level)

            # # 논문연구 수업 제외
            for i in range(1, 10):
                if i != 6: ## 논문
                    driver.find_element(By.XPATH, value=f'//*[@id="hSrchOpenSubmattFg"]/label[{i}]/em').click()
                    driver.implicitly_wait(2)

            # 검색 버튼 클릭
            driver.find_element(by=By.XPATH, value='//*[@id="HD100"]/div/div/div[3]/div[3]/button').click()
            driver.implicitly_wait(2)

            course_dict = {}

            end_crawling = False
            starting_page_num = '-1'

            while not end_crawling:

                past_starting_page_num = starting_page_num
                try: 
                    starting_page_num = driver.find_element(by=By.XPATH, value='//*[@id="skip-con"]/div/div[3]/div[4]/span/a[1]').text
                except:
                    starting_page_num = driver.find_element(by=By.XPATH, value='//*[@id="skip-con"]/div/div[3]/div[4]/span/a').text
                
                # if page is the last page
                if past_starting_page_num == starting_page_num:
                    break

                for i in range(1, 6):

                    if i != 1:
                        try:
                            driver.find_element(by=By.XPATH, value=f'//*[@id="skip-con"]/div/div[3]/div[4]/span/a[{i}]').click()
                        except:
                            # if there's no ith page button
                            end_crawling = True
                            break

                    courses = driver.find_elements(by=By.CLASS_NAME, value='course-info-item')

                    for c in courses:
                        try:
                            # 검색결과 클릭
                            #driver.find_element(by=By.XPATH, value=f'//*[@id="course_info_detail_{i}"]/div[1]').click()
                            c.click()

                            # 강의 id 수집
                            course_id = driver.find_element(by=By.XPATH, value='//*[@id="sbjtCd-ltNo"]').text

                            # 강좌명 수집
                            course_name = driver.find_element(by=By.XPATH, value='//*[@id="sbjtNm"]/strong').text
                            course_name = course_name.split('강좌명 : ')[-1].strip()

                            # 강의개요 수집
                            driver.find_element(by=By.XPATH, value='//*[@id="tab2"]/button').click()
                            abstract_kor=driver.find_element(by=By.XPATH, value='//*[@id="sbjtSmryCtnt"]').text
                            abstract_eng=driver.find_element(by=By.XPATH, value='//*[@id="sbjtSmryEngCtnt"]').text

                            course_dict[course_id] = (course_name, year, semester, query_keyword, abstract_kor, abstract_eng)

                            # 팝업창 닫기
                            driver.find_element(by=By.XPATH, value='/html/body/div[8]/div/button').click()
                        except UnexpectedAlertPresentException:
                            continue
                # next page로 이동
                driver.find_element(by=By.XPATH, value='//*[@id="skip-con"]/div/div[3]/div[4]/a[3]').click()
                driver.implicitly_wait(5)
            # 결과 없을 시 에러 처리    
            if not len(course_dict):
                continue
            # 결과 출력
            abstract_df = pd.DataFrame.from_dict(course_dict).T.reset_index()
            abstract_df.columns = ['course_id', 'course_name', 'year', 'semester', 'query_keyword', 'abstract_kor', 'abstract_eng']

            abstract_df.to_excel(f'{query_keyword}_{year}_{semester}.xlsx', index=False, encoding='utf-8-sig')
            driver.implicitly_wait(5)
            driver.quit()

In [ ]:
len(course_dict)

In [ ]:
# course_dict = {}

In [ ]:
# end_crawling = False
# starting_page_num = '-1'

In [ ]:
# while not end_crawling:

#     past_starting_page_num = starting_page_num
#     try: 
#         starting_page_num = driver.find_element(by=By.XPATH, value='//*[@id="skip-con"]/div/div[3]/div[4]/span/a[1]').text
#     except:
#         starting_page_num = driver.find_element(by=By.XPATH, value='//*[@id="skip-con"]/div/div[3]/div[4]/span/a').text
    
#     # if page is the last page
#     if past_starting_page_num == starting_page_num:
#         break

#     for i in range(1, 6):

#         if i != 1:
#             try:
#                 driver.find_element(by=By.XPATH, value=f'//*[@id="skip-con"]/div/div[3]/div[4]/span/a[{i}]').click()
#             except:
#                 # if there's no ith page button
#                 end_crawling = True
#                 break

#         courses = driver.find_elements(by=By.CLASS_NAME, value='course-info-item')

#         for c in courses:
#             try:
#                 # 검색결과 클릭
#                 #driver.find_element(by=By.XPATH, value=f'//*[@id="course_info_detail_{i}"]/div[1]').click()
#                 c.click()

#                 # 강의 id 수집
#                 course_id = driver.find_element(by=By.XPATH, value='//*[@id="sbjtCd-ltNo"]').text

#                 # 강좌명 수집
#                 course_name = driver.find_element(by=By.XPATH, value='//*[@id="sbjtNm"]/strong').text
#                 course_name = course_name.split('강좌명 : ')[-1].strip()

#                 # 강의개요 수집
#                 driver.find_element(by=By.XPATH, value='//*[@id="tab2"]/button').click()
#                 abstract_kor=driver.find_element(by=By.XPATH, value='//*[@id="sbjtSmryCtnt"]').text
#                 abstract_eng=driver.find_element(by=By.XPATH, value='//*[@id="sbjtSmryEngCtnt"]').text

#                 course_dict[course_id] = (course_name, year, semester, query_keyword, abstract_kor, abstract_eng)

#                 # 팝업창 닫기
#                 driver.find_element(by=By.XPATH, value='/html/body/div[8]/div/button').click()
#             except UnexpectedAlertPresentException:
#                 continue
#     # next page로 이동
#     driver.find_element(by=By.XPATH, value='//*[@id="skip-con"]/div/div[3]/div[4]/a[3]').click()
#     driver.implicitly_wait(5)

In [ ]:
# abstract_df = pd.DataFrame.from_dict(course_dict).T.reset_index()
# abstract_df.columns = ['course_id', 'course_name', 'year', 'semester', 'query_keyword', 'abstract_kor', 'abstract_eng']

In [ ]:
# abstract_df.to_excel(f'{query_keyword}_{year}_{semester}.xlsx', index=False, encoding='utf-8-sig')